In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


df = pd.read_csv("age_gender.csv")


def process_pixels(pixel_str):
    pixels = np.array(pixel_str.split(), dtype="uint8")
    return pixels.reshape(48, 48, 1)

df["pixels"] = df["pixels"].apply(process_pixels)


X = np.stack(df["pixels"].values) / 255.0  
y_age = df["age"].values  
y_gender = df["gender"].values 
y_ethnicity = to_categorical(df["ethnicity"].values) 


X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_ethnicity_train, y_ethnicity_test = train_test_split(
    X, y_age, y_gender, y_ethnicity, test_size=0.2, random_state=42
)


input_layer = Input(shape=(48, 48, 1))

x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)


age_output = Dense(1, activation="linear", name="age_output")(x)  
gender_output = Dense(1, activation="sigmoid", name="gender_output")(x)  
ethnicity_output = Dense(y_ethnicity.shape[1], activation="softmax", name="ethnicity_output")(x)  


model = Model(inputs=input_layer, outputs=[age_output, gender_output, ethnicity_output])


model.compile(
    optimizer="adam",
    loss={
        "age_output": "mse",  
        "gender_output": "binary_crossentropy",  
        "ethnicity_output": "categorical_crossentropy", 
    },
    metrics={
        "age_output": "mae",  
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)

# Train model
model.fit(
    X_train,
    {"age_output": y_age_train, "gender_output": y_gender_train, "ethnicity_output": y_ethnicity_train},
    validation_data=(X_test, {"age_output": y_age_test, "gender_output": y_gender_test, "ethnicity_output": y_ethnicity_test}),
    epochs=20,
    batch_size=32
)


model.save("multi_output_cnn.keras")


Epoch 1/20
593/593 ━━━━━━━━━━━━━━━━━━━━ 47s 68ms/step - age_output_loss: 290.2667 - age_output_mae: 12.7258 - ethnicity_output_accuracy: 0.3216 - ethnicity_output_loss: 5.3128 - gender_output_accuracy: 0.5686 - gender_output_loss: 2.1440 - loss: 297.7237 - val_age_output_loss: 269.1617 - val_age_output_mae: 12.3734 - val_ethnicity_output_accuracy: 0.4927 - val_ethnicity_output_loss: 1.3271 - val_gender_output_accuracy: 0.5554 - val_gender_output_loss: 0.6778 - val_loss: 271.0390
Epoch 2/20
593/593 ━━━━━━━━━━━━━━━━━━━━ 39s 66ms/step - age_output_loss: 149.3585 - age_output_mae: 9.0246 - ethnicity_output_accuracy: 0.4068 - ethnicity_output_loss: 1.5238 - gender_output_accuracy: 0.6253 - gender_output_loss: 0.6707 - loss: 151.5531 - val_age_output_loss: 338.7094 - val_age_output_mae: 15.4376 - val_ethnicity_output_accuracy: 0.4881 - val_ethnicity_output_loss: 1.3325 - val_gender_output_accuracy: 0.6100 - val_gender_output_loss: 0.6643 - val_loss: 341.0919
Epoch 3/20
593/593 ━━━━━━━━━━━━━━

In [3]:
model.save("multi_output_cnn.keras")


In [5]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("multi_output_cnn.keras")


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
import tensorflow as tf

# Load model (change filename if using .keras)
MODEL_PATH = "multi_output_cnn.keras"
model = tf.keras.models.load_model(MODEL_PATH)

# Print model summary
model.summary()


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 48, 48,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 24, 24,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 12, 12,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 12, 12,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 12, 12,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 6, 6, 128) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 4608)      │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    589,952 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ age_output (Dense)  │ (None, 1)         │        129 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gender_output       │ (None, 1)         │        129 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ethnicity_output    │ (None, 5)         │        645 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,368,400 (5.22 MB)

 Trainable params: 683,975 (2.61 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 683,977 (2.61 MB)

In [5]:
import numpy as np

# Create a dummy grayscale image (48x48)
dummy_image = np.random.rand(1, 48, 48, 1)  # Batch size 1

# Make prediction
age_pred, gender_pred, ethnicity_pred = model.predict(dummy_image)

# Process outputs
age = round(age_pred[0][0])  # Convert to integer
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
ethnicity = np.argmax(ethnicity_pred[0])  # Get class index

print(f"Predicted Age: {age}")
print(f"Predicted Gender: {gender}")
print(f"Predicted Ethnicity Class: {ethnicity}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Predicted Age: 48
Predicted Gender: Male
Predicted Ethnicity Class: 1


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path):
    img = load_img(image_path, color_mode="grayscale", target_size=(48, 48))
    img = img_to_array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Path to your test image
image_path = "your_image.jpg"
input_image = preprocess_image(image_path)

# Make prediction
age_pred, gender_pred, ethnicity_pred = model.predict(input_image)

# Process results
age = round(age_pred[0][0])
gender = "Female" if gender_pred[0][0] > 0.5 else "Male"
ethnicity = np.argmax(ethnicity_pred[0])

print(f"Predicted Age: {age}")
print(f"Predicted Gender: {gender}")
print(f"Predicted Ethnicity Class: {ethnicity}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step
Predicted Age: 51
Predicted Gender: Male
Predicted Ethnicity Class: 1


In [17]:
import numpy as np
import cv2

# Load the trained model
model = tf.keras.models.load_model("multi_output_cnn.keras")

# Function to preprocess input image
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (48, 48))  # Resize to match model input
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Expand dimensions for model input
    return img

# Predict on a test image
img_path = "mohith.jpg"  # Replace with actual image path
img = preprocess_image(img_path)
age_pred, gender_pred, ethnicity_pred = model.predict(img)

# Convert predictions to human-readable format
predicted_age = int(age_pred[0][0])
predicted_gender = "Male" if gender_pred[0][0] > 0.5 else "Female"
predicted_ethnicity = np.argmax(ethnicity_pred[0])  # Get highest probability class

print(f"Predicted Age: {predicted_age}")
print(f"Predicted Gender: {predicted_gender}")
print(f"Predicted Ethnicity Class: {predicted_ethnicity}")


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
Predicted Age: 50
Predicted Gender: Female
Predicted Ethnicity Class: 1


In [15]:
import numpy as np
import cv2
import tensorflow as tf

# Load the fine-tuned model
model = tf.keras.models.load_model("multi_output_cnn.keras")

# Define ethnicity class labels
ethnicity_classes = {0: "White", 1: "Black", 2: "Asian", 3: "Indian", 4: "Others"}

# Function to preprocess input image
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
    img = cv2.resize(img, (48, 48))  # Resize to match model input
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=[0, -1])  # Expand dimensions for model input
    return img

# Predict on a new image
def predict_image(img_path):
    img = preprocess_image(img_path)
    age_pred, gender_pred, ethnicity_pred = model.predict(img)

    # Convert predictions
    predicted_age = int(age_pred[0][0])  # Age output
    predicted_gender = "Male" if gender_pred[0][0] > 0.5 else "Female"  # Gender classification
    predicted_ethnicity = ethnicity_classes[np.argmax(ethnicity_pred[0])]  # Ethnicity classification

    print(f"Predicted Age: {predicted_age}")
    print(f"Predicted Gender: {predicted_gender}")
    print(f"Predicted Ethnicity: {predicted_ethnicity}")

# Test the script with an image
image_path = "mohith.jpg"  # Replace with actual image path
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Predicted Age: 50
Predicted Gender: Female
Predicted Ethnicity: Black


In [1]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("multi_output_cnn.keras")

# Unfreeze some layers for fine-tuning
for layer in model.layers[-10:]:  # Unfreezing last 10 layers
    if hasattr(layer, "trainable"):
        layer.trainable = True

# Recompile with a lower learning rate for fine-tuning
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower learning rate for fine-tuning
    loss={
        "age_output": "mse",
        "gender_output": "binary_crossentropy",
        "ethnicity_output": "categorical_crossentropy",
    },
    metrics={
        "age_output": "mae",
        "gender_output": "accuracy",
        "ethnicity_output": "accuracy",
    }
)


C:\Users\SHASHWATH M\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
